In [1]:
import scipy.stats as sps
import numpy_financial as npf

# Question 1

> 1.	Consider Table 2.2 in text. Suppose that income before tax is normally distributed with mean 0.5 and standard deviation 2.8. The marginal tax rate is 30%. What is the probability that the equity will be wiped out in one year? Assume that losses can be carried back to offset taxes paid in previous years. (For example, a loss of 4 would become a loss of 2.8 (= 4*(1-0.3)) after taxes are taken into account.)   

The probability that the equity will be wiped out in one year is **0.32%**. The calculation is shown below. 

In [2]:
# Equity and income target
equity = -5
tax_rate = 0.3
inc_tgt = equity / (1 - tax_rate)

# Distribution parameter
mu = 0.5
std = 2.8

# Probability of income (after tax) LOWER than target: 
prob = sps.norm.cdf(inc_tgt, mu, std)
print(f'Probability of income exceeding equity: {round((prob)*100, 2)}%')

Probability of income exceeding equity: 0.32%


\pagebreak
# Question 2

> 2. Suppose that a bank’s main activity is that it pays an average of 0.5% per annum interest on one-year deposits and makes five-year loans at a fixed rate of 3% per annum. What risks is the bank taking? How could it manage those risks using swaps?

__**Risks:**__

- **Interest rate**: Should the situation remain at status quo, the bank could benefit from a net profit margin of 2.5%. However, market interest rate for deposit may increase in the near feature, increasing the bank's cost on attracting more deposits and hence squeezing its net profit margin. 

- **Credit**: Loans that the bank lent out may default just as any other borrowing activities. Depending on the macroeconomic situation, the default rate may be higher than the bank's prior expectation. A high default rate may significantly reduce the bank's profit margin. 

- **Liquidity**: Under the current product layout, where the bank's liabilities (as represented by deposits) have a shorter maturity cycle than the bank's assets (loans). In a hypothetical scenario where a large number of clients withdraw their deposits over a short period of time, the bank may face extensive liquidity issues as most of its assets are tied into the five-year loan contracts. 

__**Swaps:**__: 

- The bank can counter the interest rate risk by engaging in **interest rate swap** and swap its fixed-rate loan interest payment for floating rate payments. 

- Similarly, credit risk can be countered using **credit default swap**, where the bank can reach an agreement to another financial institution to take ownership of the loan and compensate the bank in the event of a loan default. In return, the bank would pay regular premium to said institution.

\pagebreak
# Question 3

> 3.	From Table 3.2 in the text the probabilities of a man aged 60 , 61, and 62 dying in the next year are 0.011447, 0.012352, and 0.013248, respectively. Calculate the minimum premium an insurance company should charge for a $5 million three-year term life insurance contract issued to a man aged 60. Assume that the premium is paid at the beginning of the year and that death always takes place half way through a year. The risk-free interest rate is 6% per year (with semi-annual compounding).

Based on the above assumptions, a minimal annual premium of **$59,739.63** would need to be charged. 

In [3]:
# Mortality rate: 
death_prob = { 60: 0.011447, 61: 0.012352, 62: 0.013248}
# Other inputs: 
policy = 5*(1000000)
i = 0.06/2
loss_ratio = 0.75

# For each year - mid year payout: 
payout_60 = policy * death_prob[60]
payout_61 = policy * (1-death_prob[60]) * death_prob[61]
payout_62 = policy * (1-death_prob[60]) * (1-death_prob[61]) * death_prob[62]
payout = {60: round(payout_60,2), 61: round(payout_61,2), 62: round(payout_62,2)}

# Convert payout to start of year: 
payout_pv = npf.pv(rate=i, nper=1, fv=-payout_60, pmt=0) + \
            npf.pv(rate=i, nper=3, fv=-payout_61, pmt=0) + \
            npf.pv(rate=i, nper=5, fv=-payout_62, pmt=0)

# How many terms of premiums can we collect， discounted to PB: 
premium_term = 1 \
                + (1-death_prob[60]) * pow(1+i, -2) \
                + ((1-death_prob[60]) * (1-death_prob[61])) * pow(1+i, -4)

# Premium will be ...
premium = payout_pv/premium_term

print(f'Expected mid-year payout: ${payout}')
print(f'Expected total payout, at the time of insurance purchase: ${round(payout_pv, 2)}')
print(f'Minimal premium required, annually: ${round(premium, 2)}')

Expected mid-year payout: ${60: 57235.0, 61: 61053.03, 62: 64672.92}
Expected total payout, at the time of insurance purchase: $167227.56
Minimal premium required, annually: $59739.63


\pagebreak
# Question 4

> 4. Suppose that in a certain defined benefit pension plan: 
> - (a) Employees work for 45 years earning wages that increase at a real rate of 2%, 
> - (b) employees retire with a pension equal to 70% of their final salary and the pension increases at the rate of inflation minus 1%, 
> - (c) The pension is received for 18 year
> - (d) the pension fund’s income is invested in bonds which earn the inflation rate plus 1.5%.
>
>   Estimate the percentage of an employee’s salary that must be contributed to the pension plan if it is to remain solvent. (Hint: Do all calculations in real rather than nominal dollars.)

The employee must contribute **25.02%** of its salary to keep the pension plan solvent. Calculations can be seen below. 

*For easiness of calculation, PVG formulas are established beforehand as functions:*

In [4]:
def pvg(pmt, i, g, t): 
    '''
    Function to calculate the present value of a growing annuity (PVG), where: 
    pmt - annuity paid per term
    i - interest rate per term
    g - growth rate of payment per period
    t - number of period
    '''
    return pmt / (i - g) * (1 - ((1 + g) / (1 + i))**t)

def gfv(fv,i, g, t): 
    '''
    Function to calculate the growing annuity required to fulfill a certain future value, 
    where the growth rate of such annuity is known, where: 
    fv - future value target
    i - interest rate per term
    g - growth rate of payment per period
    t - number of period
    '''
    return fv * (i - g) / ((1 + i)**t - (1 + g)**t)

\pagebreak
Actual *calculations of Q4 can be referred here:*

In [5]:
# Assumptions
start_income = 1 # In real dollars
print(f'first year income: {start_income:.4f}')
# Given parameters: 
real_rate = 0.02
work_length = 45
pension_payout = 0.7
pension_rate = -0.01
pension_length = 18
bond_return = 0.015

# Calculation: 
# Employee's salary at his final year: 
# FV(PV=income, i=2%, t=44): - Note that the salary @ EOY1 is 1 
final_income = npf.fv(real_rate, work_length-1, pmt=0, pv=-start_income)
print(f'final_income: {final_income:.4f}')

# Annual pension payout at the starting year: income * 70%
annual_pension = final_income * pension_payout
print(f'starting annual pension: {annual_pension:.4f}')

# Total pension required: pension payout increased by pension rate and compounded by bond return
# PV(A=annual_pension, i=1.5%, g=-1%, t=18)
total_pension = pvg(pmt=annual_pension, i=bond_return, g=pension_rate, t=pension_length)
print(f'total pension payment: {total_pension:.4f}')

# Note: First pension payout should be at the beginning of the starting year,
# .... which is essentially the end of the final year: same term of final salary

# Percentage: While working, the compensation increased as salary increase and compounded by bond return
# A(FV=total_pension, i=1.5%, g=2%, t=45)
annual_contribution = gfv(fv=total_pension, i=bond_return, g=real_rate, t=work_length)
print(f'annual contribution required: {annual_contribution:.4f}')

first year income: 1.0000
final_income: 2.3901
starting annual pension: 1.6730
total pension payment: 24.2035
annual contribution required: 0.2502
